# Multi-Analyst Text Analysis Pipeline

This notebook demonstrates the full pipeline for analyzing text through multiple specialist lenses (rhetorician, syntactician, lexicologist, etc.) and synthesizing their observations.

## Installations and Preparations
First, external modules are installed and ensured to be in working order.

In [9]:
# Optional: Install requirements if running in a fresh kernel
# Uncomment if needed:
!pip install -r requirements.txt

# Or install individual packages:
# !pip install litellm pydantic jinja2


[notice] A new release of pip is available: 24.1.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [10]:
try:
    import litellm
    print('Providers\n=========')
    print('* ' + '\n* '.join(litellm.LITELLM_CHAT_PROVIDERS))
except ImportError as e:
    print(f"✗ Cannot import litellm: {e}")

Providers
* openai
* openai_like
* bytez
* xai
* custom_openai
* text-completion-openai
* cohere
* cohere_chat
* clarifai
* anthropic
* anthropic_text
* replicate
* huggingface
* together_ai
* datarobot
* openrouter
* cometapi
* vertex_ai
* vertex_ai_beta
* gemini
* ai21
* baseten
* azure
* azure_text
* azure_ai
* sagemaker
* sagemaker_chat
* bedrock
* vllm
* nlp_cloud
* petals
* oobabooga
* ollama
* ollama_chat
* deepinfra
* perplexity
* mistral
* groq
* nvidia_nim
* cerebras
* baseten
* ai21_chat
* volcengine
* codestral
* text-completion-codestral
* deepseek
* sambanova
* maritalk
* cloudflare
* fireworks_ai
* friendliai
* watsonx
* watsonx_text
* triton
* predibase
* databricks
* empower
* github
* custom
* litellm_proxy
* hosted_vllm
* llamafile
* lm_studio
* galadriel
* gradient_ai
* github_copilot
* novita
* meta_llama
* featherless_ai
* nscale
* nebius
* dashscope
* moonshot
* v0
* heroku
* oci
* morph
* lambda_ai
* vercel_ai_gateway
* wandb
* ovhcloud
* lemonade


## Initialize Base Objects
Set up connections to a Large Language Model provider via `litellm` model router. Also, setup up tools to retrieve text data to be part of the context window, that is, instruction prompts and texts to analyze. A basic result storage is also initialized.

The LLM to use is set by the `model_string`, which is constructed as `<provider>/<model>`, the providers defined by the `litellm` package, see in particular `litellm.LITELLM_CHAT_PROVIDERS`. The API key to the provider should be stored in an environment variable with name defined in `model_provider_api_key_env_var`. Do **not** store the API key as a string variable directly in the notebook.

In [11]:
model_string = 'mistral/mistral-large-2411'
model_provider_api_key_env_var = 'MISTRAL_API_KEY'

In [12]:
import os
from pathlib import Path
from belletrist import LLM, LLMConfig, PromptMaker, DataSampler, ResultStore

llm = LLM(LLMConfig(
    model=model_string,
    api_key=os.environ.get(model_provider_api_key_env_var)
))
prompt_maker = PromptMaker()
sampler = DataSampler()
store = ResultStore(Path(f"{os.getcwd()}/belletrist_storage.db"))

In case a clean run is done, the old contents of the database are discarded with a result store reset. Do not run the rest if content should be preserved.

In [13]:
store.reset()

## Generate and Store Text Samples to be Analyzed

A random text sample is taken from the data corpus and stored with full provenance (which file, which paragraphs). Each sample is an instance of `TextSegment`.

The sample size is set by the variable `n_sample` and each sample comprises `m_paragraphs_per_sample` number of consecutive paragraphs.

If non-random text samples are preferred, use the `get_paragraph_chunk` method of the `DataSampler` instance.

In [14]:
text_sample = sampler.sample_segment(p_length=4)
print(f'Text source: {text_sample.file_path}')
print(f'Paragraph range: {text_sample.paragraph_start} - {text_sample.paragraph_end}')
print(f'\n{text_sample.text}')

Text source: /Users/andersohrn/PycharmProjects/russell_writes/data/russell/mysticism_and_logic_and_other_essays.txt
Paragraph range: 529 - 533

Miss Jones[46] contends that there is no difficulty in admitting
contradictory predicates concerning such an object as "the present
King of France," on the ground that this object is in itself
contradictory. Now it might, of course, be argued that this object,
unlike the round square, is not self-contradictory, but merely
non-existent. This, however, would not go to the root of the matter.
The real objection to such an argument is that the law of
contradiction ought not to be stated in the traditional form "A is not
both B and not B," but in the form "no proposition is both true and
false." The traditional form only applies to certain propositions,
namely, to those which attribute a predicate to a subject. When the
law is stated of propositions, instead of being stated concerning
subjects and predicates, it is at once evident that propositions 

In [15]:
n_sample = 1
m_paragraphs_per_sample = 10

for _ in range(n_sample):
    sample_id = f'sample_{len(store.list_samples()) + 1:03d}'
    segment = sampler.sample_segment(p_length=m_paragraphs_per_sample)
    store.save_segment(sample_id, segment)


In [16]:
print('Sample keys:\n============')
store.list_samples()

Sample keys:


['sample_001']

## Construct the Analyst Agents and Analyze

Send the text samples through each specialist analyst. Each produces an independent analysis from their domain expertise.

**Prompt structure for caching optimization:**
1. Preamble instruction (static)
2. Analyst-specific template (static per analyst)
3. Text to analyze (dynamic)

Note that the execution of this can take time since it involves invoking LLMs. These are however independent analyses, and can therefore in principle be run in parallel, though the implementation below does not utilize that fact.

In [17]:
from belletrist.models import (
    PreambleInstructionConfig,
    PreambleTextConfig,
    RhetoricianConfig,
    SyntacticianConfig,
    LexicologistConfig,
    InformationArchitectConfig,
    EfficiencyAuditorConfig,
    CrossPerspectiveIntegratorConfig,
)
ANALYSTS = ["rhetorician", "syntactician", "lexicologist", "information_architect", "efficiency_auditor"]
ANALYST_CONFIGS = {
    "rhetorician": RhetoricianConfig,
    "syntactician": SyntacticianConfig,
    "lexicologist": LexicologistConfig,
    "information_architect": InformationArchitectConfig,
    "efficiency_auditor": EfficiencyAuditorConfig,
}

def build_analyst_prompt(preamble_instruction: str, analyst_prompt: str, preamble_text: str) -> str:
    """
    Helper function to construct the full prompt for an analyst.
    
    """
    return f"{preamble_instruction}\n\n{analyst_prompt}\n\n{preamble_text}"

In [18]:
# Get all samples from the store
all_samples = store.list_samples()
print(f"Processing {len(all_samples)} samples with {len(ANALYSTS)} analysts each\n")

# Outer loop: iterate over each text sample
for sample_id in all_samples:
    print(f"Sample: {sample_id}")
    
    # Get the sample text
    sample = store.get_sample(sample_id)
    text = sample['text']
    
    # Build shared prompt components (reused across all analysts for this sample)
    preamble_instruction = prompt_maker.render(PreambleInstructionConfig())
    preamble_text = prompt_maker.render(PreambleTextConfig(text_to_analyze=text))
    
    # Inner loop: run each analyst on this sample
    for analyst_name in ANALYSTS:
        print(f"  Running {analyst_name}...", end=" ")
        
        # Get analyst-specific prompt using the config class
        analyst_config = ANALYST_CONFIGS[analyst_name]()
        analyst_prompt = prompt_maker.render(analyst_config)
        
        # Build full prompt using helper function
        full_prompt = build_analyst_prompt(preamble_instruction, analyst_prompt, preamble_text)
        
        # Run analysis
        response = llm.complete(full_prompt)
        store.save_analysis(sample_id, analyst_name, response.content, response.model)
        
        print(f"✓ ({len(response.content)} chars)")
    
    print()  # Blank line between samples

print(f"All analyses complete for {len(all_samples)} samples")

Processing 1 samples with 5 analysts each

Sample: sample_001
  Running rhetorician... 
Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers



ServiceUnavailableError: litellm.ServiceUnavailableError: ServiceUnavailableError: MistralException - {"object":"error","message":"Service unavailable.","type":"internal_server_error","param":null,"code":"3800"}

Then verify that analysis was run as expected and yielded analysis.

In [ ]:
sample_id = 'sample_001'
is_complete = store.is_complete(sample_id, ANALYSTS)
print(f"Analysis complete: {is_complete}")

# Retrieve sample and all analyses (both are now dicts)
sample, analyses = store.get_sample_with_analyses(sample_id)

print(f"\nSample: {sample['sample_id']}")
print(f"Source: File {sample['file_index']}, paragraphs {sample['paragraph_start']}-{sample['paragraph_end']}")
print(f"Analyses available: {list(analyses.keys())}")

# Examine one analysis
print(f"\n--- Rhetorician Output (first 500 chars) ---")
print(analyses.get("rhetorician", "Not found")[:500])

## Pattern Recognition (Cross-Perspective Integration)

Synthesize all analyst perspectives to identify interactions, tensions, and load-bearing features. This is a per-text-cross-analyst transformation where a unit of writing patterns for each text sample is generated.

In [ ]:
samples_to_analyze = store.list_samples()[0:1]
print(samples_to_analyze)

In [ ]:
def build_pattern_prompt_from_(text: str, analyses: dict):
    sample, analyses = store.get_sample_with_analyses(sample_id)

    analyst_info = {}
    for analyst_name in ANALYSTS:
        config_class = ANALYST_CONFIGS[analyst_name]
        analyst_info[analyst_name] = {
            'analysis': analyses[analyst_name],
            'analyst_descr_short': config_class.description()
        }

    pattern_config = CrossPerspectiveIntegratorConfig(
        original_text=sample['text'],
        analysts=analyst_info
    )
    return prompt_maker.render(pattern_config)

In [ ]:
for sample_id in samples_to_analyze:
    print(f"Running pattern recognizer for {sample_id}...", end=" ")
    
    sample, analyses = store.get_sample_with_analyses(sample_id)
    pattern_prompt = build_pattern_prompt_from_(text=sample['text'], analyses=analyses)

    pattern_response = llm.complete(pattern_prompt)
    
    # Store pattern recognition result in result store
    store.save_analysis(
        sample_id, 
        CrossPerspectiveIntegratorConfig.analyst_name(), 
        pattern_response.content, 
        pattern_response.model
    )
    
    print(f"✓ ({len(pattern_response.content)} chars)")

In [ ]:
sample_id = 'sample_001'
sample, analyses = store.get_sample_with_analyses(sample_id)
print(analyses.keys())

print(f"\n--- Pattern Analyst Output (first 2000 chars) ---")
print(analyses.get("pattern_recognizer", "Not found")[:2000])

## Step 5: Cross-Text Synthesis (Cross-Analyst Pattern Recognition)

Now we synthesize patterns across multiple text analyses. This stage takes all the pattern recognition outputs (cross-perspective integrations from Step 4) and identifies:
- Recurring patterns across texts
- Context-dependent variations
- Hierarchies of importance
- Generalizable principles

This produces a single synthesis document that is NOT stored in the result store (since it spans multiple samples), but saved as a final output variable.

In [ ]:
from belletrist.models import CrossTextSynthesizerConfig

# Get all samples that have pattern recognition results
all_samples = store.list_samples()
pattern_analyst = CrossPerspectiveIntegratorConfig.analyst_name()

# Retrieve all pattern recognition analyses
integrated_analyses = {}
for sample_id in all_samples:
    pattern_analysis = store.get_analysis(sample_id, pattern_analyst)
    if pattern_analysis:
        integrated_analyses[sample_id] = pattern_analysis
    else:
        print(f"⚠ Sample {sample_id} missing pattern recognition analysis")

print(f"Found {len(integrated_analyses)} pattern recognition analyses")
print(f"Sample IDs: {list(integrated_analyses.keys())}")

# Only proceed if we have at least 2 (required for cross-text synthesis)
if len(integrated_analyses) >= 2:
    # Build cross-analyst synthesis prompt
    cross_analyst_config = CrossTextSynthesizerConfig(
        integrated_analyses=integrated_analyses
    )
    cross_analyst_prompt = prompt_maker.render(cross_analyst_config)
    
    # Run cross-text synthesis
    print("\\nRunning cross-text synthesis...", end=" ")
    cross_analyst_response = llm.complete(cross_analyst_prompt)
    print(f"✓ ({len(cross_analyst_response.content)} chars)")
    
    # Save to variable (not to result store - this is a final output)
    cross_text_synthesis = cross_analyst_response.content
    
    # Display first part
    print("\\n--- Cross-Text Synthesis (first 1000 chars) ---")
    print(cross_text_synthesis[:1000])
else:
    print(f"\\n⚠ Need at least 2 pattern recognition analyses for cross-text synthesis. Got {len(integrated_analyses)}.")

## Utilities: Working with Stored Samples

Helper functions for browsing and managing stored results.

In [ ]:
# List all samples in the database
all_samples = store.list_samples()
print(f"Total samples: {len(all_samples)}")
print(f"Sample IDs: {all_samples}")

# Check completion status for each
print("\nCompletion status:")
for sid in all_samples:
    complete = store.is_complete(sid, ANALYSTS)
    status = "✓" if complete else "✗"
    print(f"  {status} {sid}")

# Close database connection when done
# store.close()